In [50]:
import requests
import pydicom
from pathlib import Path
from urllib3.filepost import encode_multipart_formdata, choose_boundary
from azure.identity import DefaultAzureCredential
from typing import List

# Authentication
Set up common variables

In [46]:
dicom_service_name = "dicomservice-eastus"
path_to_dicoms_dir = "./dicoms/"

# West US
# base_url = f"https://medicalimaging-ivan-images.dicom.azurehealthcareapis.com/v1" 

# East US
base_url = f"https://healthserviceseastusws-dicomservice-eastus.dicom.azurehealthcareapis.com/v1"

study_uid = "1.2.826.0.1.3680043.8.498.13230779778012324449356534479549187420"; #StudyInstanceUID for all 3 examples
series_uid = "1.2.826.0.1.3680043.8.498.45787841905473114233124723359129632652"; #SeriesInstanceUID for green-square and red-triangle
instance_uid = "1.2.826.0.1.3680043.8.498.47359123102728459884412887463296905395"; #SOPInstanceUID for red-triangle

In [58]:
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()

print(credential.credentials) # this can be used to find the index of the AzureCliCredential

(<azure.identity._credentials.environment.EnvironmentCredential object at 0x000001C454DD8A30>, <azure.identity._credentials.managed_identity.ManagedIdentityCredential object at 0x000001C454DDA620>, <azure.identity._credentials.shared_cache.SharedTokenCacheCredential object at 0x000001C454DF0A30>, <azure.identity._credentials.azure_cli.AzureCliCredential object at 0x000001C454C503A0>, <azure.identity._credentials.azure_powershell.AzurePowerShellCredential object at 0x000001C454C505B0>)


In [59]:
# need AzureCliCredential object here
token = credential.credentials[3].get_token('https://dicom.healthcareapis.azure.com')
bearer_token = f'Bearer {token.token}'
print(token.token)

eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2RpY29tLmhlYWx0aGNhcmVhcGlzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0L2E0NjJmNDQ3LTc3Y2YtNDFiYy1hOTQ5LWQzNmI4NTg4NjM1MC8iLCJpYXQiOjE2NzcwMDUxOTYsIm5iZiI6MTY3NzAwNTE5NiwiZXhwIjoxNjc3MDA5NjYyLCJhY3IiOiIxIiwiYWlvIjoiQVZRQXEvOFRBQUFBOFNTNzZQMDlDUjFjMlFXUC8vcktMUUoyOEpwVER3LzZBU0FMRnB1MENYV0c5YzhCMWg0ZjgyZi9vd0xPOW5ZZlFwNDBCWVBsdC9LeDdxMGw4ZkxDTWI1bER1bGtZbzluZkRjMmx0emhOTDA9IiwiYWx0c2VjaWQiOiIxOmxpdmUuY29tOjAwMDM0MDAxOEFGMDIxOTYiLCJhbXIiOlsicHdkIl0sImFwcGlkIjoiMDRiMDc3OTUtOGRkYi00NjFhLWJiZWUtMDJmOWUxYmY3YjQ2IiwiYXBwaWRhY3IiOiIwIiwiZW1haWwiOiJ0YXJhcG92QGdtYWlsLmNvbSIsImZhbWlseV9uYW1lIjoiVGFyYXBvdiIsImdpdmVuX25hbWUiOiJJdmFuIiwiaWRwIjoibGl2ZS5jb20iLCJpcGFkZHIiOiI3MS4yMzEuMTc3LjUyIiwibmFtZSI6Ikl2YW4gVGFyYXBvdiIsIm9pZCI6Ijg4OGQ4YWU5LWI4MzYtNDgzMy04NmNmLTYyMWNlOTRmYjEwZiIsInB1aWQiOiIxMDAzMjAwMEM4RDNEQzJFIiwicmgiOiIwLkFWa0FSX1JpcE0

In [20]:
# Supporting method for multipart requests
def encode_multipart_related(fields, boundary=None):
    if boundary is None:
        boundary = choose_boundary()

    body, _ = encode_multipart_formdata(fields, boundary)
    content_type = str('multipart/related; boundary=%s' % boundary)

    return body, content_type

In [40]:
client = requests.session()


In [47]:
headers = {"Authorization":bearer_token}
url= f'{base_url}/changefeed'
response = client.get(url,headers=headers)

print(response.content)

b'[{"sequence":1,"partitionName":"Microsoft.Default","studyInstanceUid":"1.2.826.0.1.3680043.8.498.13230779778012324449356534479549187420","seriesInstanceUid":"1.2.826.0.1.3680043.8.498.45787841905473114233124723359129632652","sopInstanceUid":"1.2.826.0.1.3680043.8.498.47359123102728459884412887463296905395","action":"Create","timestamp":"2023-02-21T07:55:28.2364373+00:00","state":"Current","metadata":{"00080005":{"vr":"CS","Value":["ISO_IR 192"]},"00080008":{"vr":"CS","Value":["DERIVED","SECONDARY","OTHER"]},"00080016":{"vr":"UI","Value":["1.2.840.10008.5.1.4.1.1.7"]},"00080018":{"vr":"UI","Value":["1.2.826.0.1.3680043.8.498.47359123102728459884412887463296905395"]},"00080020":{"vr":"DA","Value":["20200922"]},"00080023":{"vr":"DA"},"0008002A":{"vr":"DT"},"00080030":{"vr":"TM","Value":["120000"]},"00080033":{"vr":"TM"},"00080050":{"vr":"SH"},"00080060":{"vr":"CS","Value":["OT"]},"00080064":{"vr":"CS","Value":["SYN"]},"00080090":{"vr":"PN","Value":[{"Alphabetic":"Doctor Doc"}]},"0010001

# Sending

In [54]:
def send_dicom_files(folder_name: str, path_to_dicom_dir: str, base_url: str, bearer_token: str):

    file_extension = ".*"

    # Get a list of all files in the folder with the specified file extension
    folder_path = Path(folder_name)
    file_names = [str(file_path) for file_path in folder_path.glob(f'*{file_extension}')]

     # Create a list to store the file bytes
    rawfiles = []
    
    # Read the contents of each file and add them to the list
    for file_name in file_names:
        filepath = Path(path_to_dicom_dir).joinpath(file_name)
        with open(filepath, 'rb') as reader:
            rawfile = reader.read()
        rawfiles.append(rawfile)
    
    # Create a list of tuples containing the file data and MIME type
    files = {}
    for i in range(len(rawfiles)):
        file_data = rawfiles[i]
        file_name = f'dicomfile{i}'
        file_mime_type = 'application/dicom'
        files[file_name]=('dicomfile', file_data, file_mime_type)

    # Encode the file data as multipart_related
    body, content_type = encode_multipart_related(fields=files)

    # Set the request headers
    headers = {
        'Accept': 'application/dicom+json',
        'Content-Type': content_type,
        'Authorization': bearer_token
    }

    # Send the POST request to the API endpoint
    url = f'{base_url}/studies'
    response = client.post(url, data=body, headers=headers, verify=False)

    # Return the response from the API endpoint
    return response

    # for each file in files:
        
    # filepath_red = Path(path_to_dicoms_dir).joinpath('red-triangle.dcm')
    # filepath_green = Path(path_to_dicoms_dir).joinpath('green-square.dcm')

    # # Open up and read through file and load bytes into memory 
    # with open(filepath_red,'rb') as reader:
    #     rawfile_red = reader.read()
    # with open(filepath_green,'rb') as reader:
    #     rawfile_green = reader.read()  
        
    # files = {'file_red': ('dicomfile', rawfile_red, 'application/dicom'),
    #         'file_green': ('dicomfile', rawfile_green, 'application/dicom')}

    # #encode as multipart_related
    # body, content_type = encode_multipart_related(fields = files)

    # headers = {'Accept':'application/dicom+json', "Content-Type":content_type, "Authorization":bearer_token}

    # url = f'{base_url}/studies'
    # response = client.post(url, body, headers=headers, verify=False)
    # return response

In [60]:
response = send_dicom_files(["1.dcm"], "dicoms/sample_ct", base_url, bearer_token)

In [62]:
print(f"Status: {response.status_code}\n")
print(response.content)


Status: 409

b'{"00081198":{"vr":"SQ","Value":[{"00081150":{"vr":"UI","Value":["1.2.840.10008.5.1.4.1.1.2"]},"00081155":{"vr":"UI","Value":["1.3.6.1.4.1.9590.100.1.2.407657022912537111628176213871706093007"]},"00081197":{"vr":"US","Value":[43264]},"00741048":{"vr":"SQ","Value":[{"00000902":{"vr":"LO","Value":["DICOM100: (0008,0020) - Dicom element \'StudyDate\' failed validation for VR \'DA\': Value cannot be parsed as a valid date."]}},{"00000902":{"vr":"LO","Value":["DICOM100: (0010,0030) - Dicom element \'PatientBirthDate\' failed validation for VR \'DA\': Value cannot be parsed as a valid date."]}}]}}]}}'


# Deletion